<a href="https://colab.research.google.com/github/rsadaphule/jhu-dnn/blob/main/Group_6_FINAL_SUBMISSION_6%F0%9F%8F%86%F0%9F%A6%A0Bacteria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>🦠Bacteria</font>](https://www.kaggle.com/c/3jul23jh-bacteria/rules)**. [**Instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                               # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log       # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                  # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 3jul23jh-bacteria # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log              # download competition dataset as a zip file
!unzip -o *.zip >> log                            # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show           # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 3jul23jh-bacteria
Using competition: 3jul23jh-bacteria
  teamId  teamName          submissionDate       score    
--------  ----------------  -------------------  -------  
10657298  Group 6           2023-07-16 17:35:28  0.99740  
10659328  Team 13           2023-07-16 02:19:35  0.99700  
10654664  Group 11          2023-07-15 01:26:54  0.99680  
10660016  Group 4           2023-07-15 00:41:50  0.99660  
10642678  Team 14           2023-07-15 01:53:35  0.99660  
10670163  Group 2           2023-07-14 23:47:10  0.99640  
10659652  Group 16          2023-07-15 19:36:27  0.99620  
10662540  Group 7           2023-07-16 00:51:58  0.99620  
10688653  Group 8           2023-07-16 16:26:29  0.99620  
10660342  Pratik Jasani     2023-07-16 00:31:09  0.99580  
10666245  Group 9           2023-07-16 01:21:04  0.99560  
10676664  Group 10          2023-07-16 08:11:05  0.99520  
10660940  NeilJoshi         2023-

In [ ]:
!pip -q install -U tensorflow_addons > log

In [ ]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, tensorflow_addons as tfa
from sklearn.preprocessing import PolynomialFeatures
import tensorflow as tf, tensorflow.keras as keras
from keras.layers import Flatten, Dense
# os.environ['TF_DETERMINISTIC_OPS'] = '1'; os.environ['TF_CUDNN_DETERMINISTIC'] = '1'; # allows seeding RNG on GPU
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 4.52 s, sys: 524 ms, total: 5.05 s
Wall time: 6.12 s


In [ ]:
df = pd.read_csv('XY_Bacteria.csv'); df

,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,y
0,10,32,41,39,77,122,55,81,58,31,...,38,88,80,20,36,31,32,32,10,NaN
1,10,31,52,165,225,221,150,143,48,31,...,111,125,143,159,69,45,32,71,10,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99998,10,31,66,107,142,155,142,107,66,31,...,93,66,93,93,66,31,31,31,10,4.0
99999,10,32,41,21,45,89,45,60,67,31,...,97,82,74,80,52,31,32,7,10,2.0


In [ ]:
df.info()   # observe datatypes and any missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 287 entries, A0T0G0C10 to y
dtypes: float64(1), int64(286)
memory usage: 219.0 MB


In [ ]:
vX = df.query('y!=y').drop('y', axis=1)    # slice a test sample
tXY = df.query('y==y')                     # slice training sample
tX, tY = tXY.drop('y', axis=1), tXY.y.astype(int)      # split into training I/O
vX  # test inputs
tX  # train inputs
print(tY.tolist()[:50]) # train outputs

,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
0,10,32,41,39,77,122,55,81,58,31,...,27,38,88,80,20,36,31,32,32,10
1,10,31,52,165,225,221,150,143,48,31,...,76,111,125,143,159,69,45,32,71,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9998,10,31,66,107,142,156,143,107,66,31,...,420,296,303,421,302,306,31,31,31,10
9999,10,31,66,107,142,155,142,108,66,31,...,66,429,307,93,93,66,31,31,31,10


,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
10000,10,7,48,74,119,110,127,86,66,7,...,223,319,233,361,439,368,179,174,199,10
10001,10,31,66,107,142,156,142,108,66,31,...,66,539,302,93,424,434,31,313,31,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99998,10,31,66,107,142,155,142,107,66,31,...,66,93,66,93,93,66,31,31,31,10
99999,10,32,41,21,45,89,45,60,67,31,...,52,97,82,74,80,52,31,32,7,10


[3, 3, 4, 0, 4, 3, 4, 4, 3, 3, 2, 2, 4, 2, 2, 4, 4, 2, 2, 0, 2, 3, 3, 3, 4, 3, 3, 4, 3, 3, 3, 4, 4, 2, 3, 3, 2, 0, 1, 4, 4, 2, 3, 4, 3, 0, 3, 2, 2, 0]


In [ ]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

## **Task 1. Preprocessing Pipeline**

Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

**Student's answer:**

### **FYI: Please run this notebook using GPU**

**1. Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.**

Since there are a high number of features (286) and 90k training observations, we used dimensionality reduction using PCA to reduce the input features into the model.

**2. Why did you choose these elements? (Perhaps something in EDA or prior experience lead you to these)**

 Dimensionality reduction was a major portion of our preprocessing pipeline. We determined early in the process that using all the features was time-consuming and had diminishing returns. Therefore we utilized PCA to reduce our feature dimensionality and perform feature scaling for us. Using hyperparameter tuning, we determined that the optimal number of principal components was in the range 120-130, since this value allowed us to capture 95% of the variance in the data,, while reducing the runtime substantially. After further optimisations of the model’s runtime, we decided to increase the number of components to include since this helped improve performance without exceeding the runtime.

**3. How do you evaluate the effectiveness of these elements?**

Our EDA was evaluated with the same framework with which we evaluated the models: using the training and test accuracy, plotting how this changed over the training epochs.
We also measured the loss using sparse categorical cross entropy and plotted how this changed over the training epochs.

**4 What else have you tried that worked or didn't?**

Genetic sequences sometimes contain stretches which provide no functional purpose called “junk DNA” [2], and may contain unreliable data [3,4]. With this in mind, we tried simply dropping some features with low correlation. Some features had less than .01 correlation with the dependent variable. This ended up being redundant with how effective PCA was, and was not used for time concerns.
```
corr_matrix = tXY.corr()
lowCorrelation = np.where(abs(corr_matrix['y'][:-1])<0.01)[0]
```
For each observation, we tried calculating the standard deviation as it appears that some sequences have more fluctuations than others, which might be a good signal for the model. We tested calculating the standard deviation over different windows widths of the sequences and fed this into the PCA, before using it in the model. However, this had a negligible impact on the model’s accuracy, so disregarded this approach.

We explored applying a smoothing function to the curve as we observed that each sequence exhibited volatilities at certain points in the sequence. However, after applying smoothing, and then feeding this into PCA, we noticed a drop in accuracy, so did not adopt this approach.

We used k means to cluster together similar observations, the purpose of which was to identify clusters where different classes were being clustered together. This would allow us to identify if there were particular features that provided the most/ least signals. However given the sheer number of features, it was hard to use this to derive any concrete takeaways.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from collections import Counter
import matplotlib.pyplot as plt
from keras.layers import BatchNormalization
from keras.layers import Dropout
import numpy as np, pandas as pd, tensorflow as tf, plotly.express as px, platform, plotly, os, time

# pd.set_option('display.max_rows', None)

In [ ]:
# Reduce dimensions using PCA
pca = PCA(n_components=200)
X_train_reduced = pca.fit_transform(tX)
X_test_reduced = pca.transform(vX)

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

**Student's answer:**

**1. Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.**

Our modeling approach was largely inspired by the approach outlined in the textbook [1], which involves training a baseline model (the one provided to us in the starter Colab), then iteratively identifying problems and the resultant improvements that can be made to the model, specifically referencing the solutions in this week’s chapter of the textbook. We experimented with different numbers of hidden layers, carefully balancing the size of the model with the runtime constraint of 60 seconds.

Overfitting the training dataset was a major concern for us, so we spent extra time experimenting with dropout and model regularization. Additionally, we recognized that we would need to pay special attention to the number of epochs and the use of batch normalization and principal component analysis to keep us within the afforded runtime limits. We also used he_normal initialisation as this outperformed the random normal initialisation.

We also used an exponential learning rate scheduler which reduced the learning rate at each epoch. This helped to speed up the learning by the model without overshooting at later epochs, allowing the model to converge towards a steady state performance in terms of accuracy and loss.

**2. How did these decisions guide you in modeling?**

Early on in our model development, we focused primarily on greedily improving model performance, as the baseline model was easily out-performed. This involved deepening the model architecture, varying model loss functions and optimizers, and experimenting with learning rate scheduling and regularization.

After reaching a very high performance on the public leaderboard, we determined that we were risking overfitting on the public LB, and spent more time analyzing how we might better generalize the model. This involved applying additional hyperparameter tuning utilizing 5-fold cross validation, which reduced our public leaderboard performance slightly, but gave us greater confidence that our model would generalize to the private leaderboard. In our cross validations, we evaluated the accuracy score as well as its variance as well as the loss, computing the average for these across the 5 folds.

**3. How do you evaluate the effectiveness of these elements?**

Besides our performance on Kaggle, one of our primary modes of testing the effectiveness of our models is using diagnostic plots. Two such plots were a training vs validation accuracy plot and a training vs validation loss plot. Both plots were used to gauge if we were overfitting on the training dataset, hopefully ensuring that our performance on the Private Kaggle LB is comparable to our Public Kaggle LB performance.

Another metric, albeit obvious, was closely monitoring our training time. While it is tempting to simply throw the dataset in a huge DNN, this would obviously exceed the 1 minute runtime afforded to us. So alongside the exploration of hyperparameters and their effect on accuracy, we also tracked their effect on training time.

**4. What else have you tried that worked or didn't?**

We also tested deeper neural networks, with less parameters per layer, but more layers. This ended up being more difficult to train with the ‘relu’ activation, with dropout it was able to get similar but less optimal validation accuracy compared to our final model.

We also tested other activation functions covered in the course’s textbook [1]. We tried “Leaky Relu” but it seemed to overfit the training data and had worse validation accuracy.  ELU was also tested, but it was using too much time to train compared to simple ‘relu’ with a learning rate decay. Selu was also tested but with negligible impact on accuracy.

We also tried different learning rate decays. We tried an exponential learning rate decay schedule passed to the Adam optimizer for various rates and steps, but it had slightly worse performance compared to our final model.
```
initial_learning_rate = .001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate,decay_steps=100000,decay_rate=0.98,staircase=True)
```
We also tested the model with a callback of early stop to help find more optimal models, but this did not affect model performance, so it was not included in the final model to save time.
```
# Used to early stop if more epochs is not helping
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, mode='min')
```
Finally, another suggestion from the textbook [1] was to add regularization to the model layers. We tried adding regularization layer by layer, and in combinations of layers, but we did not find this to have a noticeable impact on model performance.


![picture](https://drive.google.com/uc?export=view&id=1bBc99vHFwkLlEJ_6tHcpOxSm9cSPtI_5)

![picture2](https://drive.google.com/uc?export=view&id=1ziH0gouJ7Cq48B37JCqWJJ4FVpiLTjmn)

In [ ]:
# Set a exponential decay for the learning rate
def expDecayLR(eta0=.01, s=20):
  return lambda epoch: eta0 * 0.1**(epoch / s)  # returns a functional

In [ ]:
# Build NN model with 3 hidden layers, using Relu activation, dropout and Batch norm

# %%time
from joblib import parallel_backend

tf.random.set_seed(0)   # always seed your experiments
Init = keras.initializers.RandomNormal(seed=0)

with parallel_backend('threading'):
  m = keras.models.Sequential([
      Flatten(input_shape=[X_train_reduced.shape[1]], name='input'),
      BatchNormalization(),
      Dense(1048, activation="relu", kernel_initializer="he_normal", name='hidden1'),
      Dropout(0.4),
      BatchNormalization(),
      Dense(512, activation="relu", kernel_initializer="he_normal", name='hidden2'),
      Dropout(0.3),
      BatchNormalization(),
      Dense(512, activation="relu", kernel_initializer="he_normal", name='hidden3'),
      Dropout(0.2),
      BatchNormalization(),
      Dense(len(tY.unique()), activation='softmax', kernel_initializer=Init, name='output')])
  m.summary()
  m.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
            metrics=["accuracy"])
  hist = m.fit(X_train_reduced,
              tY,
              epochs=20,
              batch_size=1024,
              # validation_split=0.2,
              callbacks=[keras.callbacks.LearningRateScheduler(expDecayLR())])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 200)               0         
                                                                 
 batch_normalization (BatchN  (None, 200)              800       
 ormalization)                                                   
                                                                 
 hidden1 (Dense)             (None, 1048)              210648    
                                                                 
 dropout (Dropout)           (None, 1048)              0         
                                                                 
 batch_normalization_1 (Batc  (None, 1048)             4192      
 hNormalization)                                                 
                                                                 
 hidden2 (Dense)             (None, 512)               5

In [ ]:
# Plot accuracy and loss curves

# DConfig = {'modeBarButtonsToAdd':['drawline','drawopenpath','drawclosedpath','drawcircle','drawrect','eraseshape']} # more tool buttons for plotly

# print(f'Hyperparameters: ', hist.params)
# print(f'Epochs:          ', hist.epoch)
# print(f'History:         ', hist.history)
# dfHist = pd.DataFrame(hist.history, index=hist.epoch)  # create a datafrom from history dictionary
# f = px.scatter(dfHist[['accuracy', 'val_accuracy']])
# f = f.update_traces(mode='lines+markers')
# f = f.update_layout(width=800, height=400, margin=dict(l=0, r=0, t=30, b=0), title='Learning performance')
# f.show(config=DConfig)

# f2 = px.scatter(dfHist[['loss', 'val_loss']])
# f2 = f2.update_traces(mode='lines+markers')
# f2 = f2.update_layout(width=800, height=400, margin=dict(l=0, r=0, t=30, b=0), title='Learning performance')
# f2.show(config=DConfig)

In [ ]:
pY = pd.DataFrame(np.argmax(m.predict(X_test_reduced), axis=-1), index=vX.index+1, columns=['y'])
ToCSV(pY.round(0).astype(int), 'kewei_sub_final_200_pca_corrected_he_normal')
print('Observed  y distribution: ', list((tXY.y.value_counts()/len(tXY)).round(3)))                # distribution of training target level
print('Predicted y distribution: ', list((pY.round(0).astype(int).value_counts()/len(pY)).round(3))) # distribution of test target level

313/313 [==============================] - 1s 2ms/step
Observed  y distribution:  [0.303, 0.302, 0.296, 0.05, 0.048]
Predicted y distribution:  [0.304, 0.302, 0.296, 0.049, 0.049]


Below is a baseline model that produces the result on Kaggle leaderboard (LB).

In [ ]:
# %%time
# tf.random.set_seed(0)   # always seed your experiments
# Init = keras.initializers.RandomNormal(seed=0)

# tX_in = tXY.drop('y', axis=1)[:50000]
# tY_in = tXY.y[:50000]

# m = keras.models.Sequential([
#     Flatten(input_shape=[tX_in.shape[1]], name='input'),
#     Dense(50, activation="relu", kernel_initializer=Init, name='hidden'),
#     Dense(len(tY.unique()), activation='softmax', kernel_initializer=Init, name='output')])
# m.summary()
# m.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
# hist = m.fit(tX_in, tY_in, epochs=10, validation_split=0.3, batch_size=32)

In [ ]:
# pY = pd.DataFrame(np.argmax(m.predict(vX), axis=-1), index=vX.index+1, columns=['y'])
# ToCSV(pY.round(0).astype(int), '🦠Baseline')
# print('Observed  y distribution: ', list((tXY.y.value_counts()/len(tXY)).round(3)))                # distribution of training target level
# print('Predicted y distribution: ', list((pY.round(0).astype(int).value_counts()/len(pY)).round(3))) # distribution of test target level

# **References:**

1. Remember to cite your sources here as well! At the least, your textbook should be cited. Google Scholar allows you to effortlessly copy/paste an APA citation format for books and publications. Also cite StackOverflow, package documentation, and other meaningful internet resources to help your peers learn from these (and to avoid plagiarism claims).

[1] Géron, A. (2019). Hands-on machine learning with Scikit-learn, Keras, and Tensorflow: Concepts, tools, and techniques to build intelligent systems (2nd edition). O'Reilly Media.

[2] Palazzo AF, Gregory TR (2014) The Case for Junk DNA. PLoS Genet 10(5): e1004351. https://doi.org/10.1371/journal.pgen.1004351

[3] Daniels H, Jones KH, Heys S, Ford DV. Exploring the Use of Genomic and Routinely Collected Data: Narrative Literature Review and Interview Study. J Med Internet Res. 2021 Sep 24;23(9):e15739. doi: 10.2196/15739. PMID: 34559060; PMCID: PMC8501405. https://www.jmir.org/2021/9/e15739/

[4] Libbrecht, M., Noble, W. Machine learning applications in genetics and genomics. Nat Rev Genet 16, 321–332 (2015). https://doi.org/10.1038/nrg3920

[5] Wood, R. Analysis of Identification Method for Bacterial Species and Antibiotic Resistance Genes Using Optical Data From DNA Oligomers. https://www.frontiersin.org/articles/10.3389/fmicb.2020.00257/full

[6] https://machinelearningmastery.com/improve-deep-learning-performance/

[7] https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-k-fold-cross-validation-with-keras.md

[8] https://keras.io/api/layers/activations/

<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 42 sec


## 💡**Starter Ideas**

**Model**
1. Tune model hyperparameters, batch size, optimizer, NN layers

**Features**
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions and polynomial features on a small subsample. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set

**Training observations**
1. Try clustering similar observations into fewer representative observations.
1. You may also try dimension reduction methods (eg. PCA) on the transposed data matrix (if it has scaled numeric features).
1. Look for and deal with outliers or influential points in the training set
1. Deal with **imbalanced sample**: oversample smaller class, or undersample larger class, or provide observation weights or provide class weights, or seek a suitable loss function
1. Investigate distributions of features. Any missing values? Any zero values?

**Predictions**
1. Evaluate predictions and focus on poorly predicted "groups":
  1. Strongest misclassifications. E.g. the model is very confident about the wrong label
  1. Evaluate predictions near decision boundaries.

**EDA and Domain Expertise**
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Learn about the domain. Read [Analysis of Identification Method for Bacterial Species and Antibiotic Resistance Genes Using Optical Data From DNA Oligomers](https://www.frontiersin.org/articles/10.3389/fmicb.2020.00257/full) and cited/citing references.